# Example 1a: Harmonic trap - Training of DeepCalib

Example code to train DeepCalib to determine the stiffness of a Brownian particle system in a harmonic trap.

<strong>DeepCalib 1.0</strong><br/>
Enhanced force-field calibration via machine learning<br/>
version 1.0 - 27 April 2020<br/>
© Aykut Argun, Tobias Thalheim, Stefano Bo, Frank Cichos & Giovanni Volpe <br/>
[Soft Matter Lab](http://www.softmatterlab.org)

## 1. INIZIALIZATION

In [ ]:
import DeepCalib

## 2. DEFINE TRAJECTORY SIMULATION 
<a id="sec2"></a>

Here the function that simulates the motion of the Brownian particle in the force field under consideration is defined. Specifically, in this case, we consider a Brownian particle in a harmonic force field, and the motion of the particle depends on the the trap stiffness k.

This file is used to reproduce results that are shown in Fig.1 and generate the pretrained network "DeepCalib_Example 1a.h5" that is going to be needed to execute Example 1b.

Comments:
1. The function that simulates the trajectories must be called <strong>simulate_trajectory</strong>.
2. Lambda functions <strong>scale_inputs</strong>, <strong>rescale_inputs</strong>, <strong>scale_targets</strong>, and <strong>rescale_targets</strong> must also be defined. For the best performance of the learning, the rescaling of both the inputs and targets should lead to values of order 1.

In [ ]:
### Physical parameters 
from math import pi
R = 1e-7                   # Radius of the Brownian particle [m]
eta = 0.001                # Viscosity of the medium [kg m^-1 s^-1]
T = 300                    # Temperature [K]
k0 = 1e-8                  # Reference stiffness [N m^-1]
gamma0 = 2 * 6 * pi * eta * R  # Reference friction coefficient [kg s^-1]

### Simulation parameters
N = 1000                   # Number of samples of the trajectory
Dt = 1e-2                  # Timestep 
oversampling = 50          # Simulation oversampling
offset = 1000              # Number of equilibration timesteps

### Define functions to scale and rescale inputs
scale_inputs = lambda x: x * 1e+6                    # Scales input trajectory to order 1
rescale_inputs = lambda scaled_x: scaled_x * 1e-6    # Rescales input trajectory to physical units

### Define function to scale and rescale targets
from numpy import log
from numpy import exp
scale_targets = lambda k: log(k / k0)                               # Scales targets to order 1
rescale_targets = lambda scaled_k: exp(scaled_k) * k0               # Inverse of targets_scaling

### Define the simulate_trajectory function

def simulate_trajectory(batch_size=32, 
                        T=T,
                        k0=k0,
                        gamma0=gamma0,
                        N=N, 
                        Dt=Dt, 
                        oversampling=oversampling, 
                        offset=offset, 
                        scale_inputs=scale_inputs, 
                        scale_targets=scale_targets):
    
    """Simulates a Brownian particle in a harmonic trap
    
    Inputs:
    
    T:              temperature of the environment
    k0:             center of the stiffness range
    gamma0:         friction coefficient
    N:              number of trajectory data points
    Dt:             measurement period
    oversampling:   oversampling from the simulation time step (to calculate dt)
    offset:         steps of the simulation before starting to save the trajectory
    scale_inputs:   inputs scale function for the network, to normalize it comparable to 1 
    scale_targets:  targets scale function for the network, to normalize it comparable to 1
        
    Outputs:
    
    inputs: the inputs for the network, these are trajectories that have the following features: 
            
            inputs.names:          names of the input trajectory variables ('x', 'y' etc)
            inputs.values:         values of the inputs in SI units
            inputs.scalings:       short description of the scaling function for the inputs ('x*1e6' etc)
            inputs.scaled_values:  scaled values of the inputs to be passed to the network for training
            
    targets: the expected ground truth measurements for the trajectory that have following features: 
            
            targets.names:          names of the targets to be measures ('k' etc)
            targets.values:         values of the ground truth targets in SI units
            targets.scalings:       short description of the scaling function for the targets ('log(k/k0)' etc)
            targets.scaled_values:  scaled values of the ground truth targets to be passed to the network for training      
    """  

    import numpy as np
    from scipy.constants import Boltzmann as kB
    from math import pi
    from math import sqrt
    from numpy.random import randn as gauss
    from numpy.random import rand as uniform
   
    ### Randomize trajectory parameters
    
    k = k0 * (10**(uniform(batch_size) * 4 - 1.5))     # Generates random stiffness values that are uniformly distributed in log scale
    gamma = gamma0 * (uniform(batch_size) * .1 + .95)  # Marginal randomization of friction coefficient to tolarate small changes

    ### Simulate
    
    dt = Dt / oversampling                 # time step of the simulation
    x = np.zeros((batch_size, N))          # initialization of the x array
    D = kB * T / gamma                     # diffusion coefficient
    C1 = -k / gamma * dt                   # drift coefficient of the Langevin equation
    C3 = np.sqrt(2 * D * dt)               # random walk coefficient of the Langevin equation
    X = x[:, 0]
    n = 0
    
    for t in range(offset):                      # Offset (for some prerun before running)
        X = X + C1 * X + C3 * gauss(batch_size)
        
    for t in range(N * oversampling):            # Simulation                
        X = X + C1 * X + C3 * gauss(batch_size)
        if t % oversampling == 0:                # We save every oversampling^th values 
            x[:, n] = X 
            n += 1
    
    # Normalize trajectory and targets
    
    inputs = DeepCalib.trajectory(
        names=['x'],  
        values=x, 
        scalings=['x * 1e6'], 
        scaled_values=scale_inputs(x))
    
    targets = DeepCalib.targets(
        names=['k'], 
        values=k,
        scalings=['log(k / k0)'], 
        scaled_values=scale_targets(k))    
    
    return inputs, targets

## 3. CHECK TRAJECTORY SIMULATION

Checks the results of the function to simulate the trajectories by plotting some examples in rescaled units. 

Have a look at the trajectories and check if they match your system, and keep an eye on different trajectories and make sure your scaled units vary in the order of 1, i.e, neither too small (0.01 or smaller) nor too large (100 or larger)

The parameter <strong>number_of_images_to_show</strong> determines the number of trajectories that are plotted.

In [ ]:
### Show some examples of simulated trajectories

number_of_trajectories_to_show = 10
%matplotlib inline
DeepCalib.plot_sample_trajectories(simulate_trajectory, number_of_trajectories_to_show)

## 4. CREATE AND COMPILE DEEP LEARNING NETWORK

The parameters of the deep learning network are defined and the network created. The summary of the network is printed where the output shape and number of parameters for each layer can be visualized.  

Comments:
1. The parameter <strong>input_shape</strong> determines the shape of the input sequence, given by the number of time-steps times the number of samples in each input sequence. Make sure your input shape dimensions match the length of the input trajectory, in this example 2 x 500 = 1000.
2. The parmameter <strong>conv_layers_dimensions</strong> determines the number and size of LSTM layers.
3. The parameter <strong>number_of_outputs</strong> determines the number of outputs, i.e. the number of force field parameters to be estimated.

In [ ]:
### Define parameters of the deep learning network
input_shape = (2, 500)      
lstm_layers_dimensions = (1000, 250, 50)
number_of_outputs = 1

### Create deep learning network
network = DeepCalib.create_deep_learning_network(input_shape, lstm_layers_dimensions, number_of_outputs)

### Print deep learning network summary
network.summary()

## 5. TRAIN DEEP LEARNING NETWORK

The parameters for the training of the deep learning network are defined and the network is trained. The sample size, iteration number, MSE, MAE and the time of each iteration is printed.

Comments:
1. The parameter <strong>sample_sizes</strong> determines the sizes of the batches of trajectories used in the training.
2. The parameter <strong>iteration_numbers</strong> determines the numbers of batches used in the training.
3. The parameter <strong>verbose</strong> determines the frequency of the update messages. It can be either a boolean value (True/False) or a number between 0 and 1.

In [ ]:
### Define parameters of the training
sample_sizes = (8, 32, 128, 512, 2048)
iteration_numbers = (1001, 1001, 1001, 2001, 4001)
verbose = .1                    

### Training
training_history = DeepCalib.train_deep_learning_network(network, simulate_trajectory, sample_sizes, iteration_numbers, verbose)

## 6. PLOT LEARNING PERFORMANCE

The learning performance is plotted. The MSE, MAE, sample size, iteration number and iteration time are plotted against the number of timesteps. 

Comment:
1. The parameter <strong>number_of_timesteps_for_average</strong> determines the length of the average. It must be a positive integer number.

In [ ]:
### Plot learning performance
number_of_timesteps_for_average = 100
DeepCalib.plot_learning_performance(training_history, number_of_timesteps_for_average)

## 7. TEST DEEP LEARNING NETWORK ON NEW SIMULATED TRAJECTORIES

The deep learning network is tested on new simulated trajectories (parameters are defined in [section 2](#sec2)). The predicted values of the targets are plotted as function of their ground-truth values both in scaled and physical units.

Comments:
1. The parameter <strong>number_of_predictions_to_show</strong> determines the number of predictions that are shown.

In [ ]:
### Test the predictions of the deep learning network on some generated trajectories
number_of_predictions_to_show = 1000

%matplotlib inline
DeepCalib.plot_test_performance(simulate_trajectory, network, rescale_targets, number_of_predictions_to_show)

## 9. SAVE DEEP LEARNING NETWORK

Comments:
1. The parameter <strong>save_file_name</strong> is the name of the file where the deep learnign network is saved.
2. By default, the network is saved in the same folder where DeepCalib is running.

In [ ]:
save_file_name = 'Network_Example_1a.h5'
network.save(save_file_name)